# Librerias

In [70]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import kruskal
import plotly.express as px
import plotly.graph_objects as go
from sklearn.utils import resample

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
import plotly.io as pio
pio.renderers.default='notebook'

# Bases de datos

In [2]:
bd1 = pd.read_excel("./Datos/Base_datos_1.xlsx")
bd1.head()

,Tejido,Replica,Especie,Trat,PESO,As,Ca,Cu,Fe,K,Mg,Mn,P,Pb,Zn,Biomasa
0,Aereo,2,V. farnesiana,C,<0.02,0.39,12900.0,5.90,180,15200.0,980.0,28.4,1380.0,0.53,11.6,5.66
1,Aereo,10,V. farnesiana,C,<0.02,0.30,7000.0,6.24,173,17200.0,1190.0,43.1,1340.0,0.72,10.5,4.23
2,Aereo,11,V. farnesiana,C,<0.02,0.23,9200.0,5.23,172,15700.0,1900.0,39.6,1880.0,0.41,12.2,4.55
3,Radical,2,V. farnesiana,C,<0.02,11.20,31100.0,154.00,8630,11600.0,5070.0,184.0,3290.0,13.70,71.5,3.56
4,Radical,10,V. farnesiana,C,<0.02,12.30,26400.0,175.50,8150,16300.0,4670.0,157.0,3120.0,14.40,64.4,2.74


In [3]:
bd2 = pd.read_excel("./Datos/col_mico.xlsx")
bd2.head()

,Replica,Especie,Trat,Hifa,Vesícula,Arbúsculo,Espora,Endófito,Campos,% total
0,2,V. farnesiana,C,0,0,0,0,0,74,0.000000
1,5,R. communis,C,0,0,0,1,0,62,1.612903
2,8,R. communis,C,2,0,0,1,0,65,4.615385
3,10,V. farnesiana,C,1,0,0,0,0,61,1.639344
4,11,V. farnesiana,C,0,0,0,0,0,78,0.000000


# Tidy por especie

In [4]:
bd_V_farnesiana = bd1.loc[bd1['Especie'] == 'V. farnesiana'].drop(['Tejido', 'Replica', 'Especie','PESO '],axis=1)
bd_R_communis = bd1.loc[bd1['Especie'] == 'R. communis'].drop(['Tejido', 'Replica', 'Especie','PESO '],axis=1)

In [5]:
bd3 = pd.read_csv("./Datos/tabla_plantas.csv")
bd3.head()

,Especie,Tratamiento,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12,H13,H14,H15
0,V. farnesiana,C,6.0,5.5,6.5,7.0,9.0,10.5,13.0,15.0,15.5,16.0,16.0,16.0,19.0,19.0,19.5
1,V. farnesiana,C,4.0,7.0,9.5,12.0,16.0,21.0,27.0,32.5,37.5,42.0,45.0,45.0,45.5,46.0,59.0
2,V. farnesiana,C,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V. farnesiana,C,5.5,6.0,7.0,8.0,9.5,12.0,15.5,19.5,26.0,32.0,35.0,34.5,31.0,31.0,33.0
4,R. communis,C,4.0,5.5,5.5,6.5,7.5,9.5,10.5,10.5,12.5,13.0,15.0,15.5,17.5,18.0,18.5


In [6]:
bd_V_farnesianabd3 = bd3.loc[bd3['Especie'] == 'V. farnesiana']
bd_R_communisbd3 = bd3.loc[bd3['Especie'] == 'R. communis']

In [7]:
bd5 = pd.read_csv("./Datos/mortalidad_semana.csv")
bd5_V_farnesiana = bd5.loc[bd5['Especie'] == 'V. farnesiana']
bd5_R_communis = bd5.loc[bd5['Especie'] == 'R. communis']

In [8]:
bd6 = pd.read_csv("./Datos/mortalidad_semana2.csv")
bd6_V_farnesiana = bd6.loc[bd6['Especie'] == 'V. farnesiana']
bd6_R_communis = bd6.loc[bd6['Especie'] == 'R. communis']

# bd_V_farnesiana

In [60]:
# Realizar ANOVA
significancia = 0.05
# ["C","CI","J","JI","SN"]
tratamientos = ["C","CI","SN"]
for tratamiento in tratamientos:

    mi_db = bd_V_farnesiana.loc[bd_V_farnesiana['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    print('Tratamiento:{}'.format(tratamiento))

    for column in mi_db.columns:

        stat, p = stats.shapiro(mi_db[column])
        if p > significancia:
            normalidad = "Normal"
        else:
            normalidad = "No normal"
        print('\tColumna:{}\tStat= {}\tp= {}\tNormalidad:{}'.format(column,"{:.3f}".format(stat),"{:.3f}".format(p),normalidad))

    fvalue, pvalue = kruskal(bd_V_farnesiana['As'], bd_V_farnesiana['Ca'], bd_V_farnesiana['Cu'], bd_V_farnesiana['Fe'], bd_V_farnesiana['K'], bd_V_farnesiana['Mg'], bd_V_farnesiana['Mn'], bd_V_farnesiana['P'], bd_V_farnesiana['Pb'], bd_V_farnesiana['Zn'], bd_V_farnesiana['Biomasa'])
    print('Prueba de kruskal: fvalue= {}\tpvalue= {}'.format("{:.3f}".format(fvalue),"{:.3f}".format(pvalue)))

fig = px.imshow(bd_V_farnesiana.corr())
fig.show()

Tratamiento:C
	Columna:As	Stat= 0.751	p= 0.020	Normalidad:No normal
	Columna:Ca	Stat= 0.932	p= 0.599	Normalidad:Normal
	Columna:Cu	Stat= 0.747	p= 0.019	Normalidad:No normal
	Columna:Fe	Stat= 0.777	p= 0.036	Normalidad:No normal
	Columna:K	Stat= 0.862	p= 0.194	Normalidad:Normal
	Columna:Mg	Stat= 0.862	p= 0.197	Normalidad:Normal
	Columna:Mn	Stat= 0.870	p= 0.227	Normalidad:Normal
	Columna:P	Stat= 0.887	p= 0.304	Normalidad:Normal
	Columna:Pb	Stat= 0.717	p= 0.009	Normalidad:No normal
	Columna:Zn	Stat= 0.817	p= 0.083	Normalidad:Normal
	Columna:Biomasa	Stat= 0.984	p= 0.969	Normalidad:Normal
Prueba de kruskal: fvalue= 252.040	pvalue= 0.000
Tratamiento:CI
	Columna:As	Stat= 0.743	p= 0.007	Normalidad:No normal
	Columna:Ca	Stat= 0.863	p= 0.127	Normalidad:Normal
	Columna:Cu	Stat= 0.742	p= 0.007	Normalidad:No normal
	Columna:Fe	Stat= 0.836	p= 0.069	Normalidad:Normal
	Columna:K	Stat= 0.852	p= 0.099	Normalidad:Normal
	Columna:Mg	Stat= 0.818	p= 0.044	Normalidad:No normal
	Columna:Mn	Stat= 0.927	p= 0.489

In [10]:

fig = px.bar(bd_V_farnesianabd3.reset_index(), y="H1", color="Tratamiento", title="Alturas H1")
fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)
fig.show()

fig = px.bar(bd_V_farnesianabd3.reset_index(), y="H15", color="Tratamiento", title="Alturas H15")

fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)

fig.show()

fig = px.scatter(bd_V_farnesianabd3.reset_index(), y="H15", color="Tratamiento",)
#fig.add_trace(go.Scatter(y=bd3["H1"],mode='markers'))
fig.add_trace(go.Bar(y=bd_V_farnesianabd3["H1"],name='H1',))
fig.add_trace(go.Bar(y=bd_V_farnesianabd3["H15"],name='H15',))
#fig.add_trace(go.Scatter(y=bd3["H15"], color=bd3["Tratamiento"]))
fig.update_layout(title='Diferencia de Alturas')
fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)
fig.show()

In [11]:
bd5_V_farnesiana_group = bd5_V_farnesiana.groupby(['Tratamiento']).sum()
bd5_V_farnesiana_group = bd5_V_farnesiana_group.reset_index(level=0).rename(columns={"index": "Tratamiento"})
bd5_V_farnesiana_group

fig = px.bar(bd5_V_farnesiana_group, y="status",x="Tratamiento", color = "Tratamiento", title="Sobrevivencia Farnesiana")
fig.update_yaxes(
    range=(0, 11),
    constrain='domain'
)
fig.show()

bd5_V_farnesiana_group2 = bd5_V_farnesiana_group.T
bd5_V_farnesiana_group2 = bd5_V_farnesiana_group2.rename(columns=bd5_V_farnesiana_group2.iloc[0])
bd5_V_farnesiana_group2 = bd5_V_farnesiana_group2[1:-2].reset_index(level=0).rename(columns={"index": "Tratamiento"})
bd5_V_farnesiana_group2

fig = px.line(bd5_V_farnesiana_group2, x="Tratamiento", y=["C","CI","J","JI","SN"],line_shape ='spline' ,title="Alturas H15")
fig.update_yaxes(
    range=(0, 11),
    constrain='domain'
)
fig.show()

In [12]:
import plotly.graph_objects as go
fig = go.Figure()
tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:

    mi_db = bd6_V_farnesiana.loc[bd6_V_farnesiana['Tratamiento'] == tratamiento].copy()
    #print('Tratamiento:{}'.format(tratamiento))    
    
    # Ajustar el modelo de Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(mi_db['Time'], mi_db['status'],label = "{}".format(tratamiento))

    # Imprimir la tabla de supervivencia
    #kmf.plot_survival_function(show_censors=True)
    #fig = px.line(kmf.survival_function_,title="Alturas H15")

    #fig.show()
    #print(kmf.survival_function_)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index, y=kmf.survival_function_["{}".format(tratamiento)],
                    mode='lines+markers',
                    name="{}".format(tratamiento)))
    #print(mi_db)
fig.update_layout(title='Prueba de super vivencia Kaplan-Meier')
fig.update_yaxes(
    range=(0, 1),
    constrain='domain'
)
fig.show()
results = multivariate_logrank_test(bd6_V_farnesiana['Time'], bd6_V_farnesiana['Tratamiento'], bd6_V_farnesiana['status'])
print(results)

<lifelines.StatisticalResult: multivariate_logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 4
         test_name = multivariate_logrank_test

---
 test_statistic      p  -log2(p)
          15.27 <0.005      7.90


# bd_R_communis

In [59]:
# Realizar ANOVA
significancia = 0.05
# ["C","CI","J","JI","SN"]
tratamientos = ["C","CI","SN"]
for tratamiento in tratamientos:

    mi_db = bd_R_communis.loc[bd_R_communis['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    print('Tratamiento:{}'.format(tratamiento))

    for column in mi_db.columns:

        stat, p = stats.shapiro(mi_db[column])
        if p > significancia:
            normalidad = "Normal"
        else:
            normalidad = "No normal"
        print('\tColumna:{}\tStat= {}\tp= {}\tNormalidad:{}'.format(column,"{:.3f}".format(stat),"{:.3f}".format(p),normalidad))

    fvalue, pvalue = kruskal(bd_R_communis['As'], bd_R_communis['Ca'], bd_R_communis['Cu'], bd_R_communis['Fe'], bd_R_communis['K'], bd_R_communis['Mg'], bd_R_communis['Mn'], bd_R_communis['P'], bd_R_communis['Pb'], bd_R_communis['Zn'], bd_R_communis['Biomasa'])
    print('Prueba de kruskal: fvalue= {}\tpvalue= {}'.format("{:.3f}".format(fvalue),"{:.3f}".format(pvalue)))

fig = px.imshow(bd_R_communis.corr())
fig.show()

Tratamiento:C
	Columna:As	Stat= 0.857	p= 0.181	Normalidad:Normal
	Columna:Ca	Stat= 0.946	p= 0.709	Normalidad:Normal
	Columna:Cu	Stat= 0.837	p= 0.123	Normalidad:Normal
	Columna:Fe	Stat= 0.882	p= 0.280	Normalidad:Normal
	Columna:K	Stat= 0.949	p= 0.733	Normalidad:Normal
	Columna:Mg	Stat= 0.745	p= 0.018	Normalidad:No normal
	Columna:Mn	Stat= 0.871	p= 0.231	Normalidad:Normal
	Columna:P	Stat= 0.851	p= 0.161	Normalidad:Normal
	Columna:Pb	Stat= 0.920	p= 0.507	Normalidad:Normal
	Columna:Zn	Stat= 0.932	p= 0.597	Normalidad:Normal
	Columna:Biomasa	Stat= 0.944	p= 0.693	Normalidad:Normal
Prueba de kruskal: fvalue= 242.350	pvalue= 0.000
Tratamiento:CI
	Columna:As	Stat= 0.855	p= 0.107	Normalidad:Normal
	Columna:Ca	Stat= 0.953	p= 0.737	Normalidad:Normal
	Columna:Cu	Stat= 0.839	p= 0.073	Normalidad:Normal
	Columna:Fe	Stat= 0.887	p= 0.219	Normalidad:Normal
	Columna:K	Stat= 0.919	p= 0.422	Normalidad:Normal
	Columna:Mg	Stat= 0.833	p= 0.064	Normalidad:Normal
	Columna:Mn	Stat= 0.822	p= 0.048	Normalidad:No nor

In [14]:

fig = px.bar(bd_R_communisbd3.reset_index(), y="H1", color="Tratamiento", title="Alturas H1")
fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)
fig.show()

fig = px.bar(bd_R_communisbd3.reset_index(), y="H15", color="Tratamiento", title="Alturas H15")

fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)

fig.show()

fig = px.scatter(bd_R_communisbd3.reset_index(), y="H15", color="Tratamiento",)
#fig.add_trace(go.Scatter(y=bd3["H1"],mode='markers'))
fig.add_trace(go.Bar(y=bd_R_communisbd3["H1"],name='H1',))
fig.add_trace(go.Bar(y=bd_R_communisbd3["H15"],name='H15',))
#fig.add_trace(go.Scatter(y=bd3["H15"], color=bd3["Tratamiento"]))
fig.update_layout(title='Diferencia de Alturas')
fig.update_yaxes(
    range=(0, 80),
    constrain='domain'
)
fig.show()

In [15]:
bd5_R_communis_group = bd5_R_communis.groupby(['Tratamiento']).sum()
bd5_R_communis_group = bd5_R_communis_group.reset_index(level=0).rename(columns={"index": "Tratamiento"})
bd5_R_communis_group

fig = px.bar(bd5_R_communis_group, y="status",x="Tratamiento", color = "Tratamiento", title="Sobrevivencia Farnesiana")
fig.update_yaxes(
    range=(0, 11),
    constrain='domain'
)
fig.show()

bd5_R_communis_group2 = bd5_R_communis_group.T
bd5_R_communis_group2 = bd5_R_communis_group2.rename(columns=bd5_R_communis_group2.iloc[0])
bd5_R_communis_group2 = bd5_R_communis_group2[1:-2].reset_index(level=0).rename(columns={"index": "Tratamiento"})
bd5_R_communis_group2

fig = px.line(bd5_R_communis_group2, x="Tratamiento", y=["C","CI","J","JI","SN"],line_shape ='spline' ,title="Alturas H15")
fig.update_yaxes(
    range=(0, 11),
    constrain='domain'
)
fig.show()

In [16]:
import plotly.graph_objects as go
fig = go.Figure()
tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:

    mi_db = bd6_R_communis.loc[bd6_R_communis['Tratamiento'] == tratamiento].copy()
    #print('Tratamiento:{}'.format(tratamiento))    
    
    # Ajustar el modelo de Kaplan-Meier
    kmf = KaplanMeierFitter()
    kmf.fit(mi_db['Time'], mi_db['status'],label = "{}".format(tratamiento))

    # Imprimir la tabla de supervivencia
    #kmf.plot_survival_function(show_censors=True)
    #fig = px.line(kmf.survival_function_,title="Alturas H15")

    #fig.show()
    #print(kmf.survival_function_)
    fig.add_trace(go.Scatter(x=kmf.survival_function_.index, y=kmf.survival_function_["{}".format(tratamiento)],
                    mode='lines+markers',
                    name="{}".format(tratamiento)))
    #print(mi_db)
fig.update_layout(title='Prueba de super vivencia Kaplan-Meier')
fig.update_yaxes(
    range=(0, 1),
    constrain='domain'
)
fig.show()
results = multivariate_logrank_test(bd6_R_communis['Time'], bd6_R_communis['Tratamiento'], bd6_R_communis['status'])
print(results)

<lifelines.StatisticalResult: multivariate_logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 4
         test_name = multivariate_logrank_test

---
 test_statistic      p  -log2(p)
          16.07 <0.005      8.41


# Col_mico

In [17]:
bd2.groupby(['Especie','Trat']).sum().drop(["Replica","Campos","% total"],axis=1)

Hifa  Vesícula  Arbúsculo  Espora  Endófito
Especie       Trat                                             
R. communis   C       15         0          0       4         0
              CI      33         1          0      41         0
              J        7         0          0       0         1
              JI      14         0          0      12         0
              SN      45         0          1      48         3
V. farnesiana C        1         0          0       0         0
              CI      33         2          1     125         0
              J       13         0          0      13         0
              JI      17         0          0      33         1
              SN      57        10          0      92         2

In [18]:
#basura = pd.read_clipboard(sep="\t")
#basura

In [ ]:
# Realizar ANOVA
significancia = 0.05
tratamientos = ["C","CI","J","JI","SN"]
for tratamiento in tratamientos:

    mi_db = bd_V_farnesiana.loc[bd_V_farnesiana['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    print('Tratamiento:{}'.format(tratamiento))

    for column in mi_db.columns:

        stat, p = stats.shapiro(mi_db[column])
        if p > significancia:
            normalidad = "Normal"
        else:
            normalidad = "No normal"
        print('\tColumna:{}\tStat= {}\tp= {}\tNormalidad:{}'.format(column,"{:.3f}".format(stat),"{:.3f}".format(p),normalidad))

    fvalue, pvalue = kruskal(bd_V_farnesiana['As'], bd_V_farnesiana['Ca'], bd_V_farnesiana['Cu'], bd_V_farnesiana['Fe'], bd_V_farnesiana['K'], bd_V_farnesiana['Mg'], bd_V_farnesiana['Mn'], bd_V_farnesiana['P'], bd_V_farnesiana['Pb'], bd_V_farnesiana['Zn'], bd_V_farnesiana['Biomasa'])
    print('Prueba de kruskal: fvalue= {}\tpvalue= {}'.format("{:.3f}".format(fvalue),"{:.3f}".format(pvalue)))

fig = px.imshow(mi_db.corr())
fig.show()

In [23]:
bd_V_farnesiana
ejemplodf = bd_V_farnesiana.drop('Trat', axis=1)

In [35]:
ejemplodf

,As,Ca,Cu,Fe,K,Mg,Mn,P,Pb,Zn,Biomasa
0,0.39,12900.0,5.90,180,15200.0,980.0,28.4,1380.0,0.53,11.6,5.6600
1,0.30,7000.0,6.24,173,17200.0,1190.0,43.1,1340.0,0.72,10.5,4.2300
2,0.23,9200.0,5.23,172,15700.0,1900.0,39.6,1880.0,0.41,12.2,4.5500
3,11.20,31100.0,154.00,8630,11600.0,5070.0,184.0,3290.0,13.70,71.5,3.5600
4,12.30,26400.0,175.50,8150,16300.0,4670.0,157.0,3120.0,14.40,64.4,2.7400
5,13.45,20500.0,186.00,10700,11100.0,4030.0,109.5,2250.0,13.50,46.2,3.7700
12,0.51,8800.0,11.45,730,14600.0,1420.0,39.2,1640.0,1.35,13.9,5.6900
13,0.24,11000.0,5.83,102,14500.0,1410.0,71.4,1520.0,0.18,13.0,6.1900
14,0.27,12600.0,6.81,219,13400.0,1310.0,54.1,1470.0,0.35,11.2,9.0100
15,0.17,10400.0,4.55,99,17400.0,1150.0,36.8,1630.0,0.20,13.8,10.7200


# Boostrap

In [79]:
n_iterations = 10000
new_data_df = pd.DataFrame()

for i in range(n_iterations):
    newData = resample(ejemplodf, replace=True, n_samples=len(ejemplodf))
    new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
new_data_df = new_data_df.append(ejemplodf, ignore_index = True)
new_data_df.tail()

,As,Ca,Cu,Fe,K,Mg,Mn,P,Pb,Zn,Biomasa
10021,0.32,9900.0,4.77,113.0,11500.0,1310.0,46.8,1660.0,0.20,22.8,5.68
10022,14.85,13700.0,109.50,2810.0,13900.0,2670.0,116.0,1830.0,1.12,60.2,4.49
10023,12.80,15200.0,117.00,1600.0,15300.0,3540.0,93.2,1560.0,0.92,97.1,4.62
10024,12.00,10500.0,86.10,1710.0,13000.0,2730.0,85.4,2160.0,0.78,94.6,3.23
10025,13.15,11300.0,120.00,990.0,18600.0,3540.0,160.5,1800.0,0.64,208.0,3.75


## correlaciones farnesiana Con boostrap

In [76]:
n_iterations = 1000
new_data_df = pd.DataFrame()

tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:
    mi_db = bd_V_farnesiana.loc[bd_V_farnesiana['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    new_data_df = pd.DataFrame()
    #Boostrap
    for i in range(n_iterations):
        newData = resample(mi_db, replace=True, n_samples=len(mi_db))
        new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
    new_data_df = new_data_df.append(mi_db, ignore_index = True)
    fig = px.imshow(new_data_df.corr(),title="Relacion de minerales en suelo: {}".format(tratamiento))
    fig.show()


## correlaciones communis Con boostrap

In [77]:
n_iterations = 1000
new_data_df = pd.DataFrame()

tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:
    mi_db = bd_R_communis.loc[bd_R_communis['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    new_data_df = pd.DataFrame()
    #Boostrap
    for i in range(n_iterations):
        newData = resample(mi_db, replace=True, n_samples=len(mi_db))
        new_data_df = new_data_df.append(newData.mean().to_frame().T, ignore_index = True)
    new_data_df = new_data_df.append(mi_db, ignore_index = True)
    fig = px.imshow(new_data_df.corr(),title="Communis relacion de minerales en suelo: {}".format(tratamiento))
    fig.show()

## correlaciones farnesiana sin boostrap

In [68]:
# ["C","CI","SN"]
tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:
    mi_db = bd_V_farnesiana.loc[bd_V_farnesiana['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    fig = px.imshow(mi_db.corr(),title="Relacion de minerales en suelo: {}".format(tratamiento))
    fig.show()

## correlaciones communis sin boostrap

In [67]:

# ["C","CI","SN"]
tratamientos = ["C","CI","J","JI","SN"]

for tratamiento in tratamientos:
    mi_db = bd_R_communis.loc[bd_R_communis['Trat'] == tratamiento].copy().drop(["Trat"],axis=1)
    fig = px.imshow(mi_db.corr(),title="Relacion de minerales en suelo: {}".format(tratamiento))
    fig.show()

In [25]:
import pandas as pd

from sklearn.datasets import load_boston

# Loading boston dataset
boston = load_boston()

# Selecting just two fields and renaming
X = (pd.DataFrame(boston.data, columns=boston.feature_names)
     .loc[:, ["RM", "AGE"]]
     .rename(columns=dict(RM="mean_rooms_per_dwelling",
                          AGE="prop_built_prior_1940")))

y = pd.DataFrame(boston.target, columns=["median_value"])

data = pd.concat(objs=[X, y], axis=1)

In [26]:
data

,mean_rooms_per_dwelling,prop_built_prior_1940,median_value
0,6.575,65.2,24.0
1,6.421,78.9,21.6
2,7.185,61.1,34.7
3,6.998,45.8,33.4
4,7.147,54.2,36.2
...,...,...,...
501,6.593,69.1,22.4
502,6.120,76.7,20.6
503,6.976,91.0,23.9
504,6.794,89.3,22.0


In [27]:
from sklearn.linear_model import LinearRegression


# Defining number of iterations for bootstrap resample
n_iterations = 1000

# Initializing estimator
lin_reg = LinearRegression()

# Initializing DataFrame, to hold bootstrapped statistics
bootstrapped_stats = pd.DataFrame()

# Each loop iteration is a single bootstrap resample and model fit
for i in range(n_iterations):

    # Sampling n_samples from data, with replacement, as train
    # Defining test to be all observations not in train
    train = resample(data, replace=True, n_samples=len(data))
    test = data[~data.index.isin(train.index)]

    X_train = train.loc[:, ["mean_rooms_per_dwelling", "prop_built_prior_1940"]]
    y_train = train.loc[:, ["median_value"]]

    X_test = test.loc[:, ["mean_rooms_per_dwelling", "prop_built_prior_1940"]]
    y_test = test.loc[:, ["median_value"]]

    # Fitting linear regression model
    lin_reg.fit(X_train, y_train)

    # Storing stats in DataFrame, and concatenating with stats
    intercept = lin_reg.intercept_
    beta_mean_rooms_per_dwelling = lin_reg.coef_.ravel()[0]
    beta_prop_built_prior_1940 = lin_reg.coef_.ravel()[1]
    r_squared = lin_reg.score(X_test, y_test)

    bootstrapped_stats_i = pd.DataFrame(data=dict(
        intercept=intercept,
        beta_mean_rooms_per_dwelling=beta_mean_rooms_per_dwelling,
        beta_prop_built_prior_1940=beta_prop_built_prior_1940,
        r_squared=r_squared
    ))

    bootstrapped_stats = pd.concat(objs=[bootstrapped_stats,
                                         bootstrapped_stats_i])

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample

# Defining number of iterations for bootstrap resample
n_iterations = 1000

# Initializing estimator
lin_reg = LinearRegression()

# Initializing DataFrame, to hold bootstrapped statistics
bootstrapped_stats = pd.DataFrame()

# Each loop iteration is a single bootstrap resample and model fit
for i in range(n_iterations):

    # Sampling n_samples from data, with replacement, as train
    # Defining test to be all observations not in train
    train = resample(data, replace=True, n_samples=len(data))
    train
    break
    test = data[~data.index.isin(train.index)]

    X_train = train.loc[:, ["mean_rooms_per_dwelling", "prop_built_prior_1940"]]
    y_train = train.loc[:, ["median_value"]]

    X_test = test.loc[:, ["mean_rooms_per_dwelling", "prop_built_prior_1940"]]
    y_test = test.loc[:, ["median_value"]]

    # Fitting linear regression model
    lin_reg.fit(X_train, y_train)

    # Storing stats in DataFrame, and concatenating with stats
    intercept = lin_reg.intercept_
    beta_mean_rooms_per_dwelling = lin_reg.coef_.ravel()[0]
    beta_prop_built_prior_1940 = lin_reg.coef_.ravel()[1]
    r_squared = lin_reg.score(X_test, y_test)

    bootstrapped_stats_i = pd.DataFrame(data=dict(
        intercept=intercept,
        beta_mean_rooms_per_dwelling=beta_mean_rooms_per_dwelling,
        beta_prop_built_prior_1940=beta_prop_built_prior_1940,
        r_squared=r_squared
    ))

    bootstrapped_stats = pd.concat(objs=[bootstrapped_stats,
                                         bootstrapped_stats_i])

In [28]:
bootstrapped_stats

,intercept,beta_mean_rooms_per_dwelling,beta_prop_built_prior_1940,r_squared
0,-23.345936,8.133531,-0.067657,0.575537
0,-23.363144,8.093123,-0.079288,0.480295
0,-21.674921,7.795335,-0.063137,0.664809
0,-34.617180,9.703041,-0.050305,0.468607
0,-22.668014,7.779044,-0.052301,0.587044
...,...,...,...,...
0,-20.229745,7.630467,-0.073789,0.560400
0,-31.230688,9.265683,-0.060494,0.485588
0,-28.720939,8.857577,-0.069896,0.537173
0,-21.407880,7.830511,-0.071474,0.552176
